# IMDB Movie Reviews Dataset
> This Dataset contain 50k movie reviews for binary sentiment classification ( Positive Review or Negative Review )

> This 50k reviews solit into 25k training set and 25k test set

> original dataset from : http://ai.stanford.edu/~amaas/data/sentiment/

> this projects use dataset from Tensorflow Data Service

In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd

## Data

In [2]:
# load IMDB data set from tensorflow data service
import tensorflow_datasets as tfds
imdb, info = tfds.load('imdb_reviews', with_info=True , as_supervised =True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteHDWW1O/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteHDWW1O/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteHDWW1O/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
# split data into train and test set
train, test = imdb['train'], imdb['test']

print(type(train))
print(type(test))

<class 'tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter'>
<class 'tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter'>


In [4]:
train_text = []
train_label = []
test_text = []
test_label = []

for review, label in train:
  train_text.append(str(review.numpy()))
  train_label.append(label)

for review, label in test:
  test_text.append(str(review.numpy()))
  test_label.append(label)

train_label = np.array(train_label)
test_label = np.array(test_label)
print(train_label.shape)
print(test_label.shape)
print(test_text[1])

(25000,)
(25000,)
b"A blackly comic tale of a down-trodden priest, Nazarin showcases the economy that Luis Bunuel was able to achieve in being able to tell a deeply humanist fable with a minimum of fuss. As an output from his Mexican era of film making, it was an invaluable talent to possess, with little money and extremely tight schedules. Nazarin, however, surpasses many of Bunuel's previous Mexican films in terms of the acting (Francisco Rabal is excellent), narrative and theme.<br /><br />The theme, interestingly, is something that was explored again in Viridiana, made three years later in Spain. It concerns the individual's struggle for humanity and altruism amongst a society that rejects any notion of virtue. Father Nazarin, however, is portrayed more sympathetically than Sister Viridiana. Whereas the latter seems to choose charity because she wishes to atone for her (perceived) sins, Nazarin's whole existence and reason for being seems to be to help others, whether they (or we) 

## Text Processing
> Basic text processing : White Space tokenizer

In [5]:
# tokenize words in data set
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
vocab_size = 10000
#max_length = 120 # max words for each example



token = Tokenizer(num_words=vocab_size, oov_token='<Unknown>')
token.fit_on_texts(train_text)
training = token.texts_to_sequences(train_text)
testing = token.texts_to_sequences(test_text)

length = []
for i in range(len(training)):
  length.append(len(training[i]))

max_length = int(pd.Series(length).describe()['75%'])

training_final = pad_sequences(training, maxlen = max_length,padding='post', truncating='post')
testing_final = pad_sequences(testing, maxlen = max_length,padding='post', truncating = 'post')

print("Training shape :",training_final.shape)
print("label shape :", train_label.shape)
print('testing shape :', testing_final.shape)
print('testing_label shape :', test_label.shape)

Training shape : (25000, 294)
label shape : (25000,)
testing shape : (25000, 294)
testing_label shape : (25000,)


## Model binary classifier
> Use two Model, DNN and RNN

### Deep Neural Network

In [6]:
# define our model
tf.keras.backend.clear_session()
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size,32, input_length = max_length),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(120, activation='relu'),
                             tf.keras.layers.Dense(1, activation='sigmoid')

])
model.summary()

#compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 294, 32)           320000    
_________________________________________________________________
flatten (Flatten)            (None, 9408)              0         
_________________________________________________________________
dense (Dense)                (None, 120)               1129080   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 121       
Total params: 1,449,201
Trainable params: 1,449,201
Non-trainable params: 0
_________________________________________________________________


In [7]:
import datetime as dt
start = dt.datetime.today()

history = model.fit(training_final, train_label, epochs=10, validation_data= (testing_final, test_label))

finish = dt.datetime.today()
print('Process time took : ', finish-start)

Epoch 1/10
782/782 [==============================] - 9s 12ms/step - loss: 0.4056 - accuracy: 0.8005 - val_loss: 0.3229 - val_accuracy: 0.8612
Epoch 2/10
782/782 [==============================] - 9s 12ms/step - loss: 0.1144 - accuracy: 0.9601 - val_loss: 0.4563 - val_accuracy: 0.8384
Epoch 3/10
782/782 [==============================] - 9s 12ms/step - loss: 0.0174 - accuracy: 0.9953 - val_loss: 0.6409 - val_accuracy: 0.8429
Epoch 4/10
782/782 [==============================] - 9s 12ms/step - loss: 0.0025 - accuracy: 0.9996 - val_loss: 0.7347 - val_accuracy: 0.8456
Epoch 5/10
782/782 [==============================] - 9s 12ms/step - loss: 3.7655e-04 - accuracy: 1.0000 - val_loss: 0.7997 - val_accuracy: 0.8470
Epoch 6/10
782/782 [==============================] - 9s 12ms/step - loss: 1.0156e-04 - accuracy: 1.0000 - val_loss: 0.8332 - val_accuracy: 0.8473
Epoch 7/10
782/782 [==============================] - 9s 11ms/step - loss: 5.4962e-05 - accuracy: 1.0000 - val_loss: 0.8652 - val_accu

### Recurrence Neural Network

In [8]:
tf.keras.backend.clear_session()
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size,32, input_length = max_length),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
                             tf.keras.layers.Dense(120, activation='relu'),
                             tf.keras.layers.Dense(1, activation='sigmoid')

])
model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 294, 32)           320000    
_________________________________________________________________
bidirectional (Bidirectional (None, 294, 128)          49664     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense (Dense)                (None, 120)               7800      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 121       
Total params: 418,801
Trainable params: 418,801
Non-trainable params: 0
_________________________________________________________________


In [9]:
start = dt.datetime.today()

history = model.fit(training_final, train_label, epochs=10, validation_data= (testing_final, test_label))

finish = dt.datetime.today()
print('Process time took : ', finish-start)

Epoch 1/10
782/782 [==============================] - 260s 332ms/step - loss: 0.5503 - accuracy: 0.7044 - val_loss: 0.6938 - val_accuracy: 0.5003
Epoch 2/10
782/782 [==============================] - 258s 329ms/step - loss: 0.4129 - accuracy: 0.8070 - val_loss: 0.3639 - val_accuracy: 0.8428
Epoch 3/10
782/782 [==============================] - 257s 329ms/step - loss: 0.2461 - accuracy: 0.9064 - val_loss: 0.3417 - val_accuracy: 0.8510
Epoch 4/10
782/782 [==============================] - 253s 324ms/step - loss: 0.1956 - accuracy: 0.9281 - val_loss: 0.3745 - val_accuracy: 0.8575
Epoch 5/10
782/782 [==============================] - 255s 326ms/step - loss: 0.1596 - accuracy: 0.9442 - val_loss: 0.3774 - val_accuracy: 0.8562
Epoch 6/10
782/782 [==============================] - 258s 330ms/step - loss: 0.1408 - accuracy: 0.9531 - val_loss: 0.4237 - val_accuracy: 0.8473
Epoch 7/10
782/782 [==============================] - 258s 330ms/step - loss: 0.1127 - accuracy: 0.9654 - val_loss: 0.4319 -

## Text Processing
> add some extra text processing and cleaning : 
1. Whitespace Tokenizer
2. delete bad symbol from text
3. delete stopwords from text
4. Replace symbol by space in text

In [10]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
replace = re.compile('[/(){}\[\]\|@,;]')
bad_symbols = re.compile('[^0-9 a-z #+_]')
Stopwords = set(stopwords.words('english'))

In [16]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
def clean_text(texts):
  token_text = word_tokenize(texts)
  string = ' '
  text = [w for w in token_text if not w in Stopwords] # remove stopwords
  text = string.join(text)
  text = re.sub(replace, ' ', text) # replace special character with space
  text = re.sub(bad_symbols,'', text) # delete bad symbols
  return text

In [0]:
# last text container : train_text and test_text
train_clean = [clean_text(t) for t in train_text]
test_clean = [clean_text(t) for t in test_text]

In [19]:
print(train_clean[1])
print(train_text[1])

b known fall asleep films   usually due combination things including   really tired   warm comfortable sette eaten lot  owever occasion  fell asleep film rubbish  he plot development constant  onstantly slow boring  hings seemed happen   explanation causing   admit    may missed part film   watched majority everything seemed happen accord without real concern anything else   cant recommend film  
b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of what was causing them or why. I admit, I may have missed part of the film, but i watched the majority of it and everything just seemed to happen of its own accord without any real concern for anything else. I cant recommen

In [20]:

token.fit_on_texts(train_clean)
training = token.texts_to_sequences(train_clean)
testing = token.texts_to_sequences(test_clean)

length = []
for i in range(len(training)):
  length.append(len(training[i]))

max_length = int(pd.Series(length).describe()['75%'])

training_final = pad_sequences(training, maxlen = max_length,padding='post', truncating='post')
testing_final = pad_sequences(testing, maxlen = max_length,padding='post', truncating = 'post')

print("Training shape :",training_final.shape)
print("label shape :", train_label.shape)
print('testing shape :', testing_final.shape)
print('testing_label shape :', test_label.shape)

Training shape : (25000, 166)
label shape : (25000,)
testing shape : (25000, 166)
testing_label shape : (25000,)


> 3rd Quartil reduced from 294 to 166

## Model 

In [21]:
tf.keras.backend.clear_session()
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size,32, input_length = max_length),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(120, activation='relu'),
                             tf.keras.layers.Dense(1, activation='sigmoid')

])
model.summary()

#compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 166, 32)           320000    
_________________________________________________________________
flatten (Flatten)            (None, 5312)              0         
_________________________________________________________________
dense (Dense)                (None, 120)               637560    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 121       
Total params: 957,681
Trainable params: 957,681
Non-trainable params: 0
_________________________________________________________________


In [22]:
start = dt.datetime.today()

history = model.fit(training_final, train_label, epochs=10, validation_data= (testing_final, test_label))

finish = dt.datetime.today()
print('Process time took : ', finish-start)

Epoch 1/10
782/782 [==============================] - 8s 11ms/step - loss: 0.3988 - accuracy: 0.8053 - val_loss: 0.3212 - val_accuracy: 0.8626
Epoch 2/10
782/782 [==============================] - 8s 11ms/step - loss: 0.1129 - accuracy: 0.9607 - val_loss: 0.4767 - val_accuracy: 0.8350
Epoch 3/10
782/782 [==============================] - 8s 11ms/step - loss: 0.0144 - accuracy: 0.9959 - val_loss: 0.6783 - val_accuracy: 0.8337
Epoch 4/10
782/782 [==============================] - 8s 11ms/step - loss: 0.0022 - accuracy: 0.9995 - val_loss: 0.7819 - val_accuracy: 0.8370
Epoch 5/10
782/782 [==============================] - 8s 11ms/step - loss: 3.6846e-04 - accuracy: 1.0000 - val_loss: 0.8336 - val_accuracy: 0.8376
Epoch 6/10
782/782 [==============================] - 8s 11ms/step - loss: 1.0892e-04 - accuracy: 1.0000 - val_loss: 0.8667 - val_accuracy: 0.8381
Epoch 7/10
782/782 [==============================] - 8s 11ms/step - loss: 9.0101e-05 - accuracy: 1.0000 - val_loss: 0.9023 - val_accu

> total parameter reducted and become smaller and fast to train

In [23]:
model.evaluate(testing_final, test_label)

782/782 [==============================] - 2s 3ms/step - loss: 0.9950 - accuracy: 0.8387


[0.9950319528579712, 0.8386800289154053]

In [24]:
tf.keras.backend.clear_session()
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size,32, input_length = max_length),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
                             tf.keras.layers.Dense(120, activation='relu'),
                             tf.keras.layers.Dense(1, activation='sigmoid')

])
model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 166, 32)           320000    
_________________________________________________________________
bidirectional (Bidirectional (None, 166, 128)          49664     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense (Dense)                (None, 120)               7800      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 121       
Total params: 418,801
Trainable params: 418,801
Non-trainable params: 0
_________________________________________________________________


In [25]:
start = dt.datetime.today()

history = model.fit(training_final, train_label, epochs=10, validation_data= (testing_final, test_label))

finish = dt.datetime.today()
print('Process time took : ', finish-start)

Epoch 1/10
782/782 [==============================] - 153s 195ms/step - loss: 0.4203 - accuracy: 0.8016 - val_loss: 0.3428 - val_accuracy: 0.8572
Epoch 2/10
782/782 [==============================] - 152s 194ms/step - loss: 0.2528 - accuracy: 0.9042 - val_loss: 0.3376 - val_accuracy: 0.8618
Epoch 3/10
782/782 [==============================] - 152s 194ms/step - loss: 0.1836 - accuracy: 0.9328 - val_loss: 0.4091 - val_accuracy: 0.8347
Epoch 4/10
782/782 [==============================] - 152s 194ms/step - loss: 0.1439 - accuracy: 0.9484 - val_loss: 0.4774 - val_accuracy: 0.8204
Epoch 5/10
782/782 [==============================] - 152s 194ms/step - loss: 0.1191 - accuracy: 0.9584 - val_loss: 0.4838 - val_accuracy: 0.8456
Epoch 6/10
782/782 [==============================] - 152s 194ms/step - loss: 0.0758 - accuracy: 0.9742 - val_loss: 0.5195 - val_accuracy: 0.8349
Epoch 7/10
782/782 [==============================] - 152s 194ms/step - loss: 0.0645 - accuracy: 0.9790 - val_loss: 0.6452 -